In [ ]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T

In [ ]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T

# Mapping from 19+ class IDs to 6-7 macro-classes
mapping = {
    0: 0,                  # road
    1: 1, 2: 1,            # sidewalk, terrain → flat
    11: 2, 12: 2,          # person, rider → human
    13: 3, 14: 3, 15: 3, 16: 3, 17: 3, 18: 3,  # vehicle classes
    2: 4, 3: 4, 4: 4,      # building, wall, fence → construction
    5: 5, 6: 5, 7: 5,      # pole, traffic light, sign → object
    10: 6,          # sky & void → background
}


class CityscapesObstacleDataset(Dataset):
    def __init__(self, root, split='train', joint_transform=None, img_transform=None, mask_transform=None):
        self.img_dir = os.path.join(root, 'leftImg8bit_trainvaltest', 'leftImg8bit', split)
        self.gt_dir = os.path.join(root, 'gtFine_trainvaltest', 'gtFine', split)
        self.joint_transform = joint_transform
        self.img_transform = img_transform
        self.mask_transform = mask_transform

        print("🧭 Path immagini:", self.img_dir)
        print("🧭 Path maschere:", self.gt_dir)

        if not os.path.exists(self.img_dir):
            raise FileNotFoundError(f"❌ Path immagini NON trovato: {self.img_dir}")
        if not os.path.exists(self.gt_dir):
            raise FileNotFoundError(f"❌ Path maschere NON trovato: {self.gt_dir}")

        self.img_paths = []
        self.mask_paths = []

        for city in os.listdir(self.img_dir):
            img_city = os.path.join(self.img_dir, city)
            gt_city = os.path.join(self.gt_dir, city)
            for fname in os.listdir(img_city):
                if fname.endswith('_leftImg8bit.png'):
                    self.img_paths.append(os.path.join(img_city, fname))
                    label_fname = fname.replace('leftImg8bit', 'gtFine_labelIds')
                    self.mask_paths.append(os.path.join(gt_city, label_fname))

        self.obstacle_classes = [26, 27, 28, 31, 32, 33]  # es. veicoli

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert('RGB')
        mask = Image.open(self.mask_paths[idx])
        mask_np = np.array(mask)

        # 🔁 Mappatura semantica
        remapped_mask = np.full_like(mask_np, fill_value=255)
        for original, new in mapping.items():
            remapped_mask[mask_np == original] = new
        class_mask = torch.from_numpy(remapped_mask).long()

        # 🔁 Trasformazioni randomiche (una sola volta, immagine + mask)
        if self.joint_transform:
            image, class_mask = self.joint_transform(image, class_mask)

        # 🎯 Ricostruisci object_mask dopo la trasformazione
        object_mask = torch.isin(class_mask, torch.tensor(self.obstacle_classes)).long()

        # 🌈 Trasformazioni specifiche immagine e maschera
        if self.img_transform:
            image = self.img_transform(image)
        if self.mask_transform:
            class_mask = self.mask_transform(class_mask)
            object_mask = self.mask_transform(object_mask)

        return {
            'image': image,
            'class_mask': class_mask,
            'object_mask': object_mask,
            'is_ood': False
        }
